In [ ]:
class Agent:

    def __init__ (self, name):
        
        '''
        (a) Add code of object variables and message that agent was created.
        '''
    
    def check(self):
    
        '''
        (b) Add code of messages indicating the credit and holding status.
        '''
    
    def buy(self, buy_inform):
        
        '''
        (c) Add code to receive incoming information as object variables.
        Print a transaction confirmation message based on the orderable condition.
        '''
    
    def sell(self, sell_inform):
        
        '''
        (d) Add code to receive incoming information as object variables.
        Print a transaction confirmation message based on the orderable condition.
        '''

In [ ]:
'''
(e) Add code to create 'FBA_agent' object and add execute an order.
'''